# Convert the SQLite DB

The OSM data is fundamentally quite relational, and so should fit well with a relational DB (though we can expect disk usage to be high).  Python has SQLite built-in, so it's a natural choice.  (Of course, enterprise levels DBs and GIS addons would make more sense for very heavily lifting).

Here I use the Isle of Man data, from http://download.geofabrik.de/europe.html, to experiment on.

In [1]:
import sys
sys.path.insert(0, "..")

In [2]:
import sqlite3
import datetime
import os

In [3]:
import osmdigest.sqlite as sq

In [4]:
os.remove("demo.db")

In [5]:
filename = os.path.join("..", "..", "..", "Data", "isle-of-man-latest.osm.bz2")

In [6]:
start = datetime.datetime.now()
sq.convert(filename, "demo.db")

print(datetime.datetime.now() - start)

0:00:11.694735


In [7]:
db = sq.OSM_SQLite("demo.db")

In [8]:
for i, node in zip(range(10), db.nodes()):
    print(node)

Node(13802141 @ [53.5104366,-3.1942821] {})
Node(13802146 @ [53.448266,-3.0242517] {})
Node(13802147 @ [53.4165984,-3.0086659] {})
Node(13802148 @ [53.4056923,-2.9994948] {})
Node(23010534 @ [53.7770793,-3.0541303] {'highway': 'traffic_signals'})
Node(283477866 @ [54.0967845,-4.7776061] {})
Node(283477867 @ [54.2114586,-4.6541152] {})
Node(283477868 @ [54.1196968,-4.6130283] {})
Node(283477869 @ [54.222377,-4.5905299] {})
Node(283477871 @ [54.2061508,-4.5991698] {})


In [9]:
for i, node in zip(range(10), db.ways()):
    print(node)

Way(22772385 ->  [292039814, 4375877404, 4375877408, 292039921, 292039778, 13802141, 4448539080, 13802146, 472475209, 472475210, 472475212, 472475213, 472475211, 472475214, 13802147, 453404917, 453404920, 453404923, 453404925, 13802148, 621886775] {'duration': '2:45', 'name': 'Isle of Man - Liverpool', 'operator': 'Isle of Man Steam Packet Company', 'route': 'ferry'})
Way(25985909 ->  [283508770, 283480687, 1747483705, 1747483685, 1747483620, 283497137, 1747483673, 1747483715, 283515694, 283515142, 1747483793, 1747483592, 1747483782, 283485720, 1747483726, 1747483618, 283505825, 1747483814, 1747483738, 1747483727, 1747483721, 1747483706, 1747483657, 1747483713, 1747483794, 1747483788, 1747483694, 1747483684, 1747483729, 283519722, 283493902, 1747511279, 283510772, 283483149, 283499649, 283480978] {'source': 'OS Second War Revision 1940. Pub 1942', 'waterway': 'stream'})
Way(25985910 ->  [283509867, 283505044] {'name': 'Laurel Avenue', 'highway': 'residential', 'surface': 'paved', 'abut

In [10]:
for i, node in zip(range(2), db.relations()):
    print(node)

Relation(58446 ->  [Member(type='node', ref=470649732, role='label'), Member(type='node', ref=17898859, role='admin_centre'), Member(type='way', ref=28421934, role='outer'), Member(type='way', ref=28421899, role='outer'), Member(type='way', ref=28421901, role='outer'), Member(type='way', ref=144804249, role='outer'), Member(type='way', ref=160844050, role='outer'), Member(type='way', ref=229560139, role='outer'), Member(type='way', ref=141730497, role='outer'), Member(type='way', ref=142982510, role='outer'), Member(type='way', ref=191586935, role='outer'), Member(type='way', ref=191586938, role='outer'), Member(type='way', ref=147077889, role='outer'), Member(type='way', ref=179086121, role='outer'), Member(type='way', ref=147078911, role='outer'), Member(type='way', ref=141730506, role='outer'), Member(type='way', ref=140550301, role='outer'), Member(type='way', ref=369520845, role='outer'), Member(type='way', ref=139314855, role='outer'), Member(type='way', ref=139487830, role='oute

The resulting file is rather large...

In [11]:
os.stat("demo.db").st_size / 1024 / 1024

15.04296875

Let's look at the average number of tags etc.

In [12]:
import collections
collections.Counter(len(node.tags) for node in db.nodes())

Counter({0: 186864,
         1: 3609,
         2: 800,
         3: 1355,
         4: 299,
         5: 146,
         6: 66,
         7: 48,
         8: 49,
         9: 17,
         10: 13,
         11: 5,
         12: 4,
         13: 2,
         14: 2,
         15: 1,
         39: 1})

In [13]:
# Sanity check: this agrees, and is (a bit) slower
import osmdigest.digest as digest

gen = digest.parse(filename)
next(gen), next(gen)
collections.Counter(len(element.tags) for element in gen if element.name == "node")

Counter({0: 186864,
         1: 3609,
         2: 800,
         3: 1355,
         4: 299,
         5: 146,
         6: 66,
         7: 48,
         8: 49,
         9: 17,
         10: 13,
         11: 5,
         12: 4,
         13: 2,
         14: 2,
         15: 1,
         39: 1})

In [14]:
collections.Counter(len(node.tags) for node in db.ways())

Counter({0: 63,
         1: 9161,
         2: 6260,
         3: 2107,
         4: 1782,
         5: 1723,
         6: 1187,
         7: 862,
         8: 337,
         9: 195,
         10: 144,
         11: 51,
         12: 38,
         13: 17,
         14: 10,
         15: 8,
         16: 1,
         20: 1})

In [15]:
collections.Counter(len(node.tags) for node in db.relations())

Counter({0: 5,
         1: 1,
         2: 71,
         3: 21,
         4: 24,
         5: 37,
         6: 30,
         7: 8,
         8: 9,
         9: 1,
         11: 2,
         12: 2,
         13: 1,
         14: 1,
         15: 1,
         19: 1,
         21: 1,
         43: 1,
         47: 1,
         49: 1,
         137: 1,
         152: 1,
         265: 1})

In [16]:
collections.Counter(len(way.nodes) for way in db.ways())

Counter({2: 2637,
         3: 1421,
         4: 985,
         5: 6338,
         6: 1555,
         7: 2484,
         8: 1171,
         9: 1685,
         10: 522,
         11: 850,
         12: 358,
         13: 603,
         14: 274,
         15: 366,
         16: 207,
         17: 247,
         18: 182,
         19: 172,
         20: 141,
         21: 140,
         22: 94,
         23: 88,
         24: 87,
         25: 76,
         26: 49,
         27: 70,
         28: 64,
         29: 42,
         30: 54,
         31: 44,
         32: 42,
         33: 37,
         34: 38,
         35: 41,
         36: 30,
         37: 35,
         38: 16,
         39: 34,
         40: 27,
         41: 25,
         42: 26,
         43: 23,
         44: 14,
         45: 16,
         46: 19,
         47: 18,
         48: 18,
         49: 12,
         50: 16,
         51: 22,
         52: 15,
         53: 13,
         54: 9,
         55: 13,
         56: 9,
         57: 10,
         58: 10,
         59: 9

## Size of each DB component

Here we create a single DB for each component (nodes, tags, ways, ...)  Some thoughts:

- There is not much we can about nodes in terms of improving the schema.
- Tags on nodes, and everything to do with relations, form a small part of the total.
- So focus on ways and their tags.

In [25]:
def make_new_db(name):
    try:
        os.remove(name)
    except: pass
    return sqlite3.connect(name)

def close_connection(connection, name):
    connection.commit()
    connection.close()
    connection = sqlite3.connect(name, isolation_level=None)
    connection.execute("vacuum")
    connection.close()
    return os.stat(name).st_size / 1024 / 1024

In [26]:
connection = make_new_db("nodes.db")
connection.execute("create table nodes(osm_id integer primary key, longitude integer, latitude integer)")
for res in db.connection.execute("select * from nodes"):
    connection.execute("insert into nodes values(?,?,?)", tuple(res))
close_connection(connection, "nodes.db")

3.72265625

In [27]:
connection = make_new_db("node_tags.db")
connection.execute("create table node_tags(osm_id integer, key text, value text)")
connection.execute("create index node_tags_osm_id_idx on node_tags(osm_id)")
for res in db.connection.execute("select * from node_tags"):
    connection.execute("insert into node_tags values(?,?,?)", tuple(res))
close_connection(connection, "node_tags.db")

0.58203125

In [28]:
connection = make_new_db("ways.db")
connection.execute("create table ways(osm_id integer, position integer, noderef integer)")
connection.execute("create index ways_idx on ways(osm_id, position)")
for res in db.connection.execute("select * from ways"):
    connection.execute("insert into ways values(?,?,?)", tuple(res))
close_connection(connection, "ways.db")

7.66796875

In [29]:
connection = make_new_db("way_tags.db")
connection.execute("create table way_tags(osm_id integer, key text, value text)")
connection.execute("create index way_tags_osm_id_idx on way_tags(osm_id)")
for res in db.connection.execute("select * from way_tags"):
    connection.execute("insert into way_tags values(?,?,?)", tuple(res))
close_connection(connection, "way_tags.db")

2.7890625

In [30]:
connection = make_new_db("relations.db")
connection.execute("create table relations(osm_id integer, member text, memberref integer, role text)")
connection.execute("create index relations_idx on relations(osm_id)")
for res in db.connection.execute("select * from relations"):
    connection.execute("insert into relations values(?,?,?,?)", tuple(res))
close_connection(connection, "relations.db")

0.2109375

In [31]:
connection = make_new_db("relation_tags.db")
connection.execute("create table relation_tags(osm_id integer, key text, value text)")
connection.execute("create index relation_tags_osm_id_idx on relation_tags(osm_id)")
for res in db.connection.execute("select * from relation_tags"):
    connection.execute("insert into relation_tags values(?,?,?)", tuple(res))
close_connection(connection, "relation_tags.db")

0.08203125

Change the schema for ways.

In [41]:
def chunk(nodes, length):
    i = 0
    out = []
    part = []
    while i < len(nodes):
        part.append(nodes[i])
        i += 1
        if i == len(nodes):
            while len(part) < length:
                part.append(None)
            out.append(part)
            break
        if len(part) == length:
            out.append(part)
            part = []
    return out
    
assert(chunk([1,2,3,4,5,6], 2) == [[1,2],[3,4],[5,6]])
assert(chunk([1,2,3,4,5], 2) == [[1,2],[3,4],[5,None]])

In [46]:
length = 10

connection = make_new_db("ways_trial.db")
create_string = ("create table ways(osm_id integer, position integer,"
    + ", ".join(["noderef"+str(i)+" integer" for i in range(length)])
    + ")" )
insert_string = ("insert into ways values (?,?,"
    + ",".join(["?"]*length) + ")" )
connection.execute(create_string)
connection.execute("create index ways_idx on ways(osm_id, position)")
for way in db.ways():
    for pos, part in enumerate(chunk(way.nodes, length)):
        data = (way.osm_id, pos, *part)
        connection.execute(insert_string, data)
close_connection(connection, "ways.db")

7.66796875

In [50]:
conn = sqlite3.connect("ways_trial.db")
